In [ ]:
import adaptive_latents as al

import matplotlib.pyplot as plt
import numpy as np

from prediction_regression_run import pred_reg_run_with_defaults
rng = np.random.default_rng()

In [ ]:
run_without_logging = pred_reg_run_with_defaults('leventhal24u', log_level=1)

In [ ]:
run_with_logging = pred_reg_run_with_defaults('odoherty21', log_level=2)


In [ ]:
runs_by_bubbles = dict()

# runs_by_bubbles[800] = pred_reg_run_with_defaults('odoherty21', log_level=1, n_bubbles=800)
# runs_by_bubbles[500] = pred_reg_run_with_defaults('odoherty21', log_level=1, n_bubbles=500)
runs_by_bubbles[1100] = run_without_logging
# runs_by_bubbles[1400] = pred_reg_run_with_defaults('odoherty21', log_level=1, n_bubbles=1400)
# runs_by_bubbles[1700] = pred_reg_run_with_defaults('odoherty21', log_level=1, n_bubbles=1700)



In [ ]:
%matplotlib inline


stream_label_mapping = {
    0: 'neural sample',
    1: 'behavior sample',
    2: 'target data sample',
    3: 'prediction',
    # -1: 'thrown out sample',
}


def time_scatter(ax, transformer, run, semilogy=True):
    a = al.timed_data_source.ArrayWithTime.from_list(transformer.log['step_time'])*1000
    stream = np.array(transformer.log['stream'])
    for s in stream_label_mapping:
        sub_a = a[stream == s]
        sub_a.t = sub_a.t[stream==s]
        ax.plot(sub_a.t, sub_a, '.', label=stream_label_mapping[s], alpha=1 if s == -1 else 1)

    q = np.quantile(a,.999)
    # ax.axhline(q, color='k')
    # ax.text(0, q*1.2, f'{q:.1f} ms')
    sampling_dt = run.sources[0][0].dt * 1000
    ax.axhline(sampling_dt, color='k', lw=1.5, ls='-')
    ax.set_xlabel('experiment time (s)')
    ax.set_ylabel('end-to-end iteration time (ms)')
    ax.set_title(f'step times for a "{type(transformer).__name__}" step on the {run.dataset} dataset')
    ax.legend()
    if semilogy:
        ax.semilogy()
        ax.text(0, sampling_dt*1.2, f'{sampling_dt:.1f} ms (sampling rate)')
    else:
        ax.set_ylim([0, sampling_dt*2])
        ax.text(0, sampling_dt + .2, f'{sampling_dt:.1f} ms (sampling rate)')


fig, axs = plt.subplots(ncols=2, figsize=[15, 5])
time_scatter(axs[0], run_without_logging.pipeline, run_without_logging, semilogy=False)
time_scatter(axs[1], run_with_logging.pipeline, run_with_logging, semilogy=False)



In [ ]:
%matplotlib inline

def row_hist_per_stream(transformer, run):
    fig, axs = plt.subplots(nrows=len(stream_label_mapping), squeeze=False, sharex=True)
    
    
    a = al.timed_data_source.ArrayWithTime.from_list(transformer.log['step_time'])*1000
    stream = np.array(transformer.log['stream'])
    sampling_dt = run.sources[0][0].dt * 1000


    for idx, s in enumerate(stream_label_mapping):
        sub_a = a[stream == s]
        sub_a.t = sub_a.t[stream==s]
        if sub_a.size:
            l,r = np.quantile(sub_a,.005), np.quantile(sub_a,.995)
            axs[idx,0].hist(sub_a, bins=np.linspace(l,r,101), color=f'C{idx}')
            axs[idx,0].semilogy()

        # axs[idx,0].axvline(sampling_dt, color='k', lw=1.5, ls='-')
        # axs[idx,0].set_xlim([0, sampling_dt*2])

    # ax.text(0, q*1.2, f'{q:.1f} ms')
    # ax.text(0, sampling_dt*1.2, f'{sampling_dt:.1f} ms (sampling rate)')
    axs[0,0].set_title('processing step length by stream')
    axs[-1,0].set_xlabel('step time (ms)')
    
run = run_without_logging
row_hist_per_stream(run.pipeline.steps[1], run)

In [ ]:

print(run.pipeline.steps[1])


In [ ]:

def row_hist_per_step(pipeline, run, s):
    fig, axs = plt.subplots(nrows=len(pipeline.steps), squeeze=False, sharex=True, figsize=(4,10))
    sampling_dt = run.sources[0][0].dt * 1000

    for idx, transformer in enumerate(pipeline.steps):
            a = al.timed_data_source.ArrayWithTime.from_list(transformer.log['step_time'])*1000
            stream = np.array(transformer.log['stream'])
            sub_a = a[stream == s]
            sub_a.t = sub_a.t[stream==s]
            if sub_a.size:
                l,r = np.quantile(sub_a,.0005), np.quantile(sub_a,.9995)
                axs[idx,0].hist(sub_a, bins=np.linspace(l,r,101), color=f'C{idx}')
                axs[idx,0].semilogy()
                # axs[idx,0].axvline(sampling_dt, color='k', lw=1.5, ls='-', alpha=.1)
                axs[idx,0].text(.98,.65, type(transformer).__name__, horizontalalignment='right', transform=axs[idx,0].transAxes)


    axs[0,0].set_title('processing time by step for predictions')
    axs[-1,0].set_xlabel('step time (ms)')

run = run_without_logging
row_hist_per_step(run.pipeline, run, 0)


In [ ]:
bw_times = al.timed_data_source.ArrayWithTime.from_list(run_without_logging.pipeline.steps[-2].log['step_time'])
reg_times = al.timed_data_source.ArrayWithTime.from_list(run_without_logging.pipeline.steps[-1].log['step_time'])


In [ ]:
%matplotlib qt
fig, ax = plt.subplots()
streams = np.array(run_without_logging.pipeline.steps[-1].log['stream'])
for stream in stream_label_mapping:
    ax.scatter(bw_times[streams==stream], reg_times[streams==stream], s=3)
# ax.semilogy()
# ax.semilogx()
plt.axis('equal')


In [ ]:
%matplotlib inline
    

bubbles = []
times = []
fig, ax = plt.subplots()
for n_bubbles, run in runs_by_bubbles.items():
    transformer = run.pipeline.steps[7]
    step_times = np.array(transformer.log['step_time'])
    streams = np.array(transformer.log['stream'])
    subset_times = step_times[streams==0]
    ax.scatter(n_bubbles + rng.normal(size=subset_times.shape)*2, subset_times * 1000, s=1, color='C0')
    
    
ax.set_ylim([0, 125])
ax.set_xlabel('number of bubbles')
ax.set_ylabel('pipeline time per neural step (ms)')
ax.set_title('Scaling of speed with number of bubbles')


# Environment comparison

In [ ]:
import pickle
import adaptive_latents as al
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
with open(al.CONFIG.bwrun_save_path / 'profile.pkl', 'rb') as fhan:
    p_m = pickle.load(fhan)

with open(al.CONFIG.bwrun_save_path / 'profile_vc.pkl', 'rb') as fhan:
    p_s = pickle.load(fhan)

with open(al.CONFIG.bwrun_save_path / 'profile_with_stuff.pkl', 'rb') as fhan:
    p_l = pickle.load(fhan)

    


In [ ]:
p_s.keys()

In [ ]:
k = 'bubblewrap'

plt.hist(p_s[k]*1000, np.linspace(0,60,101), label='in virtual console');
plt.hist(p_m[k]*1000, np.linspace(0,60,101), label='command line, after reboot');
plt.hist(p_l[k]*1000, np.linspace(0,60,101), label='jupyter notebook, with other things open');
plt.legend()
# plt.semilogx()
# plt.semilogy()